In [ ]:
'''
# W O R K   F L O W  #
1. download github data in native clickhouse format (74.6 gb, ~10hours to download)
2. clickhouse server must be running
see: https://clickhouse.tech/docs/en/getting-started/install/
 >sudo service clickhouse-server start (may need sudo -u japple)
 >clickhouse-client
# Insert the database into clickhouse
3. create the db tables:
 >CREATE TABLE github_events ...
# see https://github-sql.github.io/explorer/#install-clickhouse
4. Insert the DB file into clickhouse <E:\Documents\Clickhouse Github data\github_events_v2.native.xz>
# (~2-300 gb? takes ~4 hours to insert)
5. run code here to connect to clickhouse client and manipulate data
#
# Note the clickhouse driver (python) communicates with the clickhouse server via a native TCP/IP protocol 
# that ships data as typed values; this will cause problems when INSERT-ing into a DB, however I don't see
# this as an issue
'''


In [1]:
from sqlalchemy import create_engine
from clickhouse_driver import Client
# dependencies
# >ipython-sql
# install by command prompt:
# >conda install -yc conda-forge ipython-sql
client = Client('localhost')

In [ ]:
# load CSV file into dataframe
# get test dataframe with different repos
#  loop through dataframe
#   pull repo
#   build query
#   run query
#   write to dataframe

In [2]:
import pandas as pd
# not yet needed here
import sys,time
import math

In [3]:
# Read CSV file into DataFrame df
# 200_repos_ready.csv has no index, CMC_id is in first column
# NaN is assigned to empty cells
dfs = pd.read_csv('200_repos.csv', index_col=0)

In [4]:
df = dfs[['repo','forge']].copy()

In [ ]:
# subset dataframes for testing
# use .copy() as slicing will not allow for assignment
df10 = df.iloc[:10].copy()
df33 = df.iloc[:33].copy()

In [9]:
# ----------------------------------------------------
# ----S T A R S --------------------------------------
# ----------------------------------------------------

query_stars_L = '''
SELECT 
    count() 
FROM github_events 
WHERE event_type = 'WatchEvent' 
    AND repo_name ='''
query_stars_R = '''
GROUP BY action
'''
repo = '''
'HuobiGroup/huobi-eco-chain' 
'''

In [203]:
query_test_noStars = '''
SELECT 
    count() 
FROM github_events 
WHERE event_type = 'WatchEvent' 
    AND repo_name =
'millecodex/SEM'
GROUP BY action
'''

In [ ]:
query2 = '''
SELECT 
    count() 
FROM github_events 
WHERE event_type = 'WatchEvent' 
    AND repo_name =
'HuobiGroup/huobi-eco-chain'
GROUP BY action
'''

In [206]:
res=client.execute(query_test_noStars)
if not res: print('not')

not


In [ ]:
# test query that returns empty list (no results)
if not res2: print('not')

In [16]:
# Write a function for this
#
# initialize new column to null/None
df['stars']=None
# iterate the dataframe as follows:
'''
loop through dataframe
  pull repo
  build query
  run query
  update dataframe
'''
num = 0
num_u = 0
for row in df.itertuples():
    # only github for now as client is connected to github_events DB
    if row.forge == 'github':
        stars = 0
        repo = row.repo
        # skip the NaN repos
        if type(repo) == str:
            query = query_stars_L + '\''+repo+'\'' + query_stars_R
            stars = client.execute(query)
            num += 1
            # query returns a tuple of list elements accessible by [first list][first item]
            # no stars returns an empty list
            if not stars:
                df.at[row.Index, 'stars'] = 0
            else: 
                df.at[row.Index, 'stars'] = stars[0][0]
                num_u += 1
                
print(num, 'repos queried and', num_u, 'STARS stats updated')

168 repos queried and 138 STARS updated


In [18]:
# write update to 200_copy_stars.csv
df.to_csv('200_stars.csv', encoding='utf-8')

In [19]:
# Read in 200_repos.csv 
# has no index, CMC_id is in first column
dfr = pd.read_csv('200_repos.csv', index_col=0)
# new df with only 2 columns
# 'CMC_id' as index is maintained
df = dfr[['repo','forge']].copy()


In [20]:
# ----------------------------------------------------
# ----F O R K S --------------------------------------
# ----------------------------------------------------

query_forks_L = '''
SELECT 
    count() AS forks 
FROM github_events 
WHERE event_type = 'ForkEvent' AND repo_name =
'''
query_forks_R = '''
'curvefi/curve-dao-contracts/tree/master/doc'
'''
query_forks = query_forks_L + query_forks_R
query_forks

"\nSELECT \n    count() AS forks \nFROM github_events \nWHERE event_type = 'ForkEvent' AND repo_name =\n\n'curvefi/curve-dao-contracts/tree/master/doc'\n"

In [ ]:
result=client.execute(query_forks)
print(result)

In [22]:
# Write a function for this
#
# initialize new column to null/None
# might not be necessary
df['forks']=None
# iterate the dataframe as follows:
'''
loop through dataframe
  pull repo
  build query
  run query
  update dataframe
'''
num = 0
num_u = 0
for row in df.itertuples():
    # only github for now as client is connected to github_events DB
    if row.forge == 'github':
        forks = 0
        repo = row.repo
        # skip the NaN repos
        if type(repo) == str:
            query = query_forks_L + '\''+repo+'\''
            forks = client.execute(query)
            num += 1
            # query returns a tuple of list elements accessible by [first list][first item]
            # no forks returns an empty list
            if not forks:
                df.at[row.Index, 'forks'] = 0
            else: 
                df.at[row.Index, 'forks'] = forks[0][0]
                num_u += 1
print(num, 'repos queried and', num_u, 'FORKS stats updated')

168 repos queried and 168 FORKS stats updated


In [23]:
# write update to 200_forks.csv
df.to_csv('200_forks.csv', encoding='utf-8', index=1)
df

,repo,forge,forks
CMC_id,,,
1,bitcoin/bitcoin,github,26356
1027,ethereum/go-ethereum,github,12090
1839,binance-chain/bsc,github,25
52,ripple/rippled,github,1390
825,NaN,NaN,None
...,...,...,...
6841,NaN,NaN,NaN
7617,NaN,NaN,NaN
5802,NaN,NaN,NaN


In [24]:
# merge two csv files into one
# 1. 200_stars.csv
# 2. 200_forks.csv
#
# might prefer to append the new column? merge seems a bit cumbersome?
#
# has no index, CMC_id is in first column
dfs = pd.read_csv('200_stars.csv', index_col = 0)
#dfsm = dfs[['stars']].copy()
dff = pd.read_csv('200_forks.csv', index_col = 0)
#dffm = dff[['forks']].copy()
#

In [29]:
# 'CMC_id' is the key, however 'repo', and 'forge' are also merged
# to prevent duplicate columns
# -> might be uncecessary?
dfm = pd.merge(dfs, dff, on = ['CMC_id','repo','forge'], how = 'outer')
dfm.drop(['repo', 'forge'], axis = 1, inplace = True)


In [31]:
dfin = pd.read_csv('200_merged.csv', index_col = 0)
dfin

,CMC_rank,name,ticker,source_code,check_source,notes,repo,forge,criticality,web_primary,web_secondary,alexa_rank
CMC_id,,,,,,,,,,,,
1,1.0,Bitcoin,BTC,https://github.com/bitcoin/bitcoin,y,NaN,bitcoin/bitcoin,github,0.86864,https://bitcoin.org/,NaN,4955.0
1027,2.0,Ethereum,ETH,https://github.com/ethereum/go-ethereum,y,NaN,ethereum/go-ethereum,github,0.82297,https://www.ethereum.org/,NaN,5558.0
1839,3.0,Binance Coin,BNB,https://github.com/binance-chain/bsc,y,fork of go-ethereum,binance-chain/bsc,github,NaN,https://www.binance.com/,NaN,141.0
52,4.0,XRP,XRP,https://github.com/ripple/rippled,y,NaN,ripple/rippled,github,0.63159,https://xrpl.org/,NaN,89113.0
825,5.0,Tether,USDT,private,y,NaN,NaN,NaN,NaN,https://tether.to,NaN,38719.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6841,NaN,Phala Network,PHA,NaN,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7617,NaN,saffron.finance,SFI,NaN,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5802,NaN,Sora,XOR,NaN,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
dfm2 = pd.merge(dfin, dfm, on = ['CMC_id'], how = 'outer')
dfm2

,CMC_rank,name,ticker,source_code,check_source,notes,repo,forge,criticality,web_primary,web_secondary,alexa_rank,stars,forks
CMC_id,,,,,,,,,,,,,,
1,1.0,Bitcoin,BTC,https://github.com/bitcoin/bitcoin,y,NaN,bitcoin/bitcoin,github,0.86864,https://bitcoin.org/,NaN,4955.0,53646.0,26356.0
1027,2.0,Ethereum,ETH,https://github.com/ethereum/go-ethereum,y,NaN,ethereum/go-ethereum,github,0.82297,https://www.ethereum.org/,NaN,5558.0,30598.0,12090.0
1839,3.0,Binance Coin,BNB,https://github.com/binance-chain/bsc,y,fork of go-ethereum,binance-chain/bsc,github,NaN,https://www.binance.com/,NaN,141.0,55.0,25.0
52,4.0,XRP,XRP,https://github.com/ripple/rippled,y,NaN,ripple/rippled,github,0.63159,https://xrpl.org/,NaN,89113.0,4135.0,1390.0
825,5.0,Tether,USDT,private,y,NaN,NaN,NaN,NaN,https://tether.to,NaN,38719.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6841,NaN,Phala Network,PHA,NaN,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7617,NaN,saffron.finance,SFI,NaN,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5802,NaN,Sora,XOR,NaN,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# write update to 200_merged.csv
dfm2.to_csv('200_merged.csv', encoding = 'utf-8', index = 1)

In [35]:
# ----------------------------------------------------
# ----A U T H O R S-----------------------------------
# ----------------------------------------------------
# A most-recent three-month average 
# excluding current month because it is in progress
# modify for static clickhouse data which stops at 2020-12-07
# >>created_at >= dateSub(MONTH, 6,toStartOfMonth(now())) AND
# >>created_at < dateSub(MONTH, 3,toStartOfMonth(now()))
#
QUERY_AUTHORS = '''
SELECT
    ROUND( SUM(authors) / COUNT(month), 2) AS average
FROM
(
    SELECT 
        uniq(actor_login) AS authors,
        toMonth(created_at) AS month,
        toYear(created_at) AS year
    FROM github_events
    WHERE event_type IN ('PullRequestEvent', 'IssuesEvent', 'IssueCommentEvent', 'PullRequestReviewCommentEvent') AND
        repo_name = 'bitcoin/bitcoin' AND
        created_at >= dateSub(MONTH, 3,toStartOfMonth(now())) AND
        created_at < toStartOfMonth(now())
    GROUP BY month, year
    ORDER BY year DESC, month DESC
)'''
query_authors_L = '''
SELECT
    ROUND( SUM(authors) / COUNT(month), 2) AS average
FROM
(
    SELECT 
        uniq(actor_login) AS authors,
        toMonth(created_at) AS month,
        toYear(created_at) AS year
    FROM github_events
    WHERE event_type IN ('PullRequestEvent', 'IssuesEvent', 'IssueCommentEvent', 'PullRequestReviewCommentEvent') AND
        repo_name = 
'''
q_repo='bitcoin/bitcoin'
query_authors_R = '''AND
        /*created_at >= dateSub(MONTH, 3,toStartOfMonth(now())) AND
        created_at < toStartOfMonth(now())*/
        created_at >= dateSub(MONTH, 6,toStartOfMonth(now())) AND
        created_at < dateSub(MONTH, 3,toStartOfMonth(now()))
    GROUP BY month, year
    ORDER BY year DESC, month DESC
)'''
query_authors=query_authors_L + '\'' + q_repo + '\'' + query_authors_R

In [36]:
# Read in 200_repos.csv
dfr = pd.read_csv('200_repos.csv', index_col=0)
# new df with only 2 columns
# 'CMC_id' as index is maintained
df = dfr[['repo','forge']].copy()
#dfs = df[0:20].copy()

In [92]:
res=client.execute(QUERY_AUTHORS)
res

[(130.5,)]

In [ ]:
print(QUERY_AUTHORS)

In [93]:
print(query_authors)

SELECT
    ROUND( SUM(authors) / COUNT(*) ,2) AS average
FROM
(
    SELECT 
        uniq(actor_login) AS authors,
        toMonth(created_at) AS month,
        toYear(created_at) AS year
    FROM github_events
    WHERE event_type IN ('PullRequestEvent', 'IssuesEvent', 'IssueCommentEvent', 'PullRequestReviewCommentEvent') AND
        repo_name = 
'bitcoin/bitcoin'AND
        created_at > date_sub(MONTH, 7, now()) AND
        created_at < date_sub(MONTH, 4, now())
    GROUP BY month, year
    ORDER BY year DESC, month DESC
)


In [38]:
num = 0
num_u = 0
for row in df.itertuples():
    # only github for now as client is connected to github_events DB
    if row.forge == 'github':
        #forks = 0
        repo = row.repo
        # skip the NaN repos
        if type(repo) == str:
            query = query_authors_L + '\'' + repo + '\'' + query_authors_R
            authors = client.execute(query)
            num += 1
            # query returns a tuple of list elements accessible by [first list][first item]
            # average of no authors returns a nan
            if math.isnan(authors[0][0]):
                df.at[row.Index, 'authors'] = 0
            else: 
                df.at[row.Index, 'authors'] = authors[0][0]
                num_u += 1
                
print(num, 'repos queried and', num_u, 'AUTHORS stats updated')

168 repos queried and 126 AUTHORS stats updated


In [40]:
# write update to 200_authors.csv
df.to_csv('200_authors.csv', encoding = 'utf-8', index = 1)

In [41]:
# update MERGED sheet with new data
# 'CMC_id' is the key, however 'repo', and 'forge' are also merged
# to prevent duplicate columns
df_temp = pd.read_csv('200_merged.csv', index_col = 0)
dfm = pd.merge(df_temp, df, on=['CMC_id', 'repo', 'forge'], how = 'outer')
dfm.to_csv('200_merged.csv', encoding = 'utf-8', index = 1)

In [42]:
# ----------------------------------------------------
# ----C O M M I T S-----------------------------------
# ----------------------------------------------------
# A most-recent three-month average 
# excluding current month because it is in progress
#
# modify for static clickhouse data which stops at 2020-12-07:
# >>created_at >= dateSub(MONTH, 6,toStartOfMonth(now())) AND
# >>created_at < dateSub(MONTH, 3,toStartOfMonth(now()))
# 
# note: there will be moderate timezone discrepancies, especially 
#       when calculating near the first of the month
#
QUERY_COMMITS = '''
SELECT ROUND( SUM(sum_push_distinct) / COUNT(month), 2) AS average
FROM
(
    SELECT SUM(push_distinct_size) AS sum_push_distinct, 
        toMonth(created_at) AS month,
        toYear(created_at) AS year
    FROM github_events
    WHERE repo_name = 'bitcoin/bitcoin' AND 
        event_type = 'PushEvent' AND
        /*created_at >= dateSub(MONTH, 3,toStartOfMonth(now())) AND
        created_at < toStartOfMonth(now())*/
        created_at >= dateSub(MONTH, 7,toStartOfMonth(now())) AND
        created_at < dateSub(MONTH, 4,toStartOfMonth(now()))
    GROUP BY month, year
    ORDER BY year DESC, month DESC
)
'''
query_commits_L ='''
SELECT ROUND( SUM(sum_push_distinct) / COUNT(month), 2) AS average
FROM
(
    SELECT SUM(push_distinct_size) AS sum_push_distinct, 
        toMonth(created_at) AS month,
        toYear(created_at) AS year
    FROM github_events
    WHERE repo_name = 
'''
q_repo='bitcoin/bitcoin'
query_commits_R = '''
AND 
        event_type = 'PushEvent' AND
        /*created_at >= dateSub(MONTH, 3,toStartOfMonth(now())) AND
        created_at < toStartOfMonth(now())*/
        created_at >= dateSub(MONTH, 7,toStartOfMonth(now())) AND
        created_at < dateSub(MONTH, 4,toStartOfMonth(now()))
    GROUP BY month, year
    ORDER BY year DESC, month DESC
)
'''
query_commits=query_commits_L + '\'' + q_repo + '\'' + query_commits_R

In [163]:
res=client.execute(query_commits)
res

[(365.0,)]

In [43]:
# Read in 200_repos.csv
dfr = pd.read_csv('200_repos.csv', index_col=0)
# new df with only 2 columns
# 'CMC_id' as index is maintained
df = dfr[['repo','forge']].copy()

In [181]:
query_test_zero='''
SELECT ROUND( SUM(sum_push_distinct) / COUNT(month), 2) AS average
FROM
(
    SELECT SUM(push_distinct_size) AS sum_push_distinct, 
        toMonth(created_at) AS month,
        toYear(created_at) AS year
    FROM github_events
    WHERE repo_name = 'Uniswap/uniswap-v2-core' AND 
        event_type = 'PushEvent' AND
        /*created_at >= dateSub(MONTH, 3,toStartOfMonth(now())) AND
        created_at < toStartOfMonth(now())*/
        created_at >= dateSub(MONTH, 6,toStartOfMonth(now())) AND
        created_at < dateSub(MONTH, 3,toStartOfMonth(now()))
    GROUP BY month, year
    ORDER BY year DESC, month DESC
)'''
res=client.execute(query_test_zero)
res

[(nan,)]

In [ ]:
import math
if math.isnan(res[0][0]): print('not')
else: print('dunno')


In [45]:
num = 0
num_u = 0
for row in df.itertuples():
    # only github for now as client is connected to github_events DB
    if row.forge == 'github':
        #forks = 0
        repo = row.repo
        # skip the NaN repos
        if type(repo) == str:
            query = query_commits_L + '\'' + repo + '\'' + query_commits_R
            result = client.execute(query)
            num += 1
            # query returns a tuple of list elements accessible by [first list][first item]
            # average of no commits returns a nan
            if math.isnan(result[0][0]):
                df.at[row.Index, 'commits'] = 0
            else: 
                df.at[row.Index, 'commits'] = result[0][0]
                num_u += 1
print(num, 'repos queried and', num_u, 'COMMITS stats updated')

168 repos queried and 125 COMMITS stats updated


In [46]:
# write update to 200_commits.csv
df.to_csv('200_commits.csv', encoding='utf-8', index=1)

In [47]:
# update MERGED sheet with new data
# 'CMC_id' is the key, however 'repo', and 'forge' are also merged
# to prevent duplicate columns
df_temp = pd.read_csv('200_merged.csv', index_col = 0)
dfm = pd.merge(df_temp, df, on = ['CMC_id', 'repo', 'forge'], how = 'inner')
dfm.to_csv('200_merged.csv', encoding='utf-8', index=1)

In [48]:
# ----------------------------------------------------
# ----C O M M E N T S---------------------------------
# ----------------------------------------------------
# total COMMENTS includes all commenting activity
# any comments counts as activity and increase engagement
# there are 3 event_type comment events:
# >CommitCommentEvent
# >IssueCommentEvent
# >CommitCommentEvent
#
'''
/* View distribution of comments*/
SELECT 
    uniq(comment_id) AS total_comments,
    uniqIf(comment_id, event_type = 'PullRequestReviewCommentEvent') AS pr_comments,
    uniqIf(comment_id, event_type = 'IssueCommentEvent') AS issue_comments,
    uniqIf(comment_id, event_type = 'CommitCommentEvent') AS commit_comments,
    toMonth(created_at) AS month,
    toYear(created_at) AS year
FROM github_events
WHERE 
   repo_name = 'bitcoin/bitcoin' AND
   toYear(created_at) >= 2020
GROUP BY month, year
ORDER BY year DESC, month DESC
'''
# only Sept/Oct/Nov 2020 #
QUERY_COMMENTS='''
SELECT ROUND( SUM(total) / COUNT(month), 2) AS average
FROM
(
SELECT 
    (
        uniqIf(comment_id, event_type = 'PullRequestReviewCommentEvent')+
        uniqIf(comment_id, event_type = 'IssueCommentEvent')+
        uniqIf(comment_id, event_type = 'CommitCommentEvent') ) AS total,
    toMonth(created_at) AS month,
    toYear(created_at) AS year
FROM github_events
WHERE 
   repo_name = 'bitcoin/bitcoin' AND
   /*created_at >= dateSub(MONTH, 3,toStartOfMonth(now())) AND
   created_at < toStartOfMonth(now())*/
   created_at >= dateSub(MONTH, 7,toStartOfMonth(now())) AND
   created_at < dateSub(MONTH, 4,toStartOfMonth(now()))
GROUP BY month, year
ORDER BY year DESC, month DESC
)
'''
query_L='''
SELECT ROUND( SUM(total) / COUNT(month), 2) AS average
FROM
(
SELECT 
    (
        uniqIf(comment_id, event_type = 'PullRequestReviewCommentEvent')+
        uniqIf(comment_id, event_type = 'IssueCommentEvent')+
        uniqIf(comment_id, event_type = 'CommitCommentEvent') ) AS total,
    toMonth(created_at) AS month,
    toYear(created_at) AS year
FROM github_events
WHERE 
   repo_name = 
'''
query_R='''
AND
   /*created_at >= dateSub(MONTH, 3,toStartOfMonth(now())) AND
   created_at < toStartOfMonth(now())*/
   created_at >= dateSub(MONTH, 7,toStartOfMonth(now())) AND
   created_at < dateSub(MONTH, 4,toStartOfMonth(now()))
GROUP BY month, year
ORDER BY year DESC, month DESC
)
'''

In [209]:
res=client.execute(QUERY_COMMENTS)
res

[(3107.33,)]

In [52]:
#
query_L='''
SELECT ROUND( SUM(total) / COUNT(month), 2) AS average
FROM
(
SELECT 
    (
        uniqIf(comment_id, event_type = 'PullRequestReviewCommentEvent')+
        uniqIf(comment_id, event_type = 'IssueCommentEvent')+
        uniqIf(comment_id, event_type = 'CommitCommentEvent') ) AS total,
    toMonth(created_at) AS month,
    toYear(created_at) AS year
FROM github_events
WHERE 
   repo_name = 
'''
query_R='''
AND
   /*created_at >= dateSub(MONTH, 3,toStartOfMonth(now())) AND
   created_at < toStartOfMonth(now())*/
   created_at >= dateSub(MONTH, 7,toStartOfMonth(now())) AND
   created_at < dateSub(MONTH, 4,toStartOfMonth(now()))
GROUP BY month, year
ORDER BY year DESC, month DESC
)
'''
'''
@column name of the column to be added to the dataframe
@query_L
@query_R
@df dataframe
'''
def runQuery(column_name, query_L, query_R, df):
    num = 0
    num_u = 0
    for row in df.itertuples():
        # only github for now as client is connected to github_events DB
        if row.forge == 'github':
            repo = row.repo
            # skip the NaN repos
            if type(repo) == str:
                query = query_L + '\'' + repo + '\'' + query_R
                result = client.execute(query)
                num += 1
                # query returns a tuple of list elements accessible by [first list][first item]
                # average of zero returns a nan
                if math.isnan(result[0][0]):
                    df.at[row.Index, column_name] = 0
                else: 
                    df.at[row.Index, column_name] = result[0][0]
                    num_u += 1
        # proof-of-life
        sys.stdout.write(".")
        sys.stdout.flush()
    output = str(num) + ' repos queried and ' + str(num_u) + ' ' + column_name + ' stats updated '
    return output

In [50]:
# Read in 200_repos.csv
dfr = pd.read_csv('200_repos.csv', index_col=0)
# new df with only 2 columns
# 'CMC_id' as index is maintained
df = dfr[['repo','forge']].copy()

In [53]:
runQuery('comments', query_L, query_R,df)

...............................................................................................................................................................................................................................

'168 repos queried and 140 comments stats updated '

In [54]:
# update MERGED sheet with new data
# 'CMC_id' is the key, however 'repo', and 'forge' are also merged
# to prevent duplicate columns
df_temp = pd.read_csv('200_merged.csv', index_col=0)
dfm = pd.merge(df_temp,df,on=['CMC_id','repo','forge'])
dfm.to_csv('200_merged.csv', encoding='utf-8', index=1)

In [55]:
# view all PR activity sorted into: opened, closed, reopened
'''
SELECT  COUNT() AS total,
    SUM(action = 'opened') AS opened,
    SUM(action = 'closed') AS closed,
    SUM(action = 'reopened') AS reopened,
    toYear(created_at) AS year, 
    toMonth(created_at) AS month
FROM github_events
WHERE repo_name = 'bitcoin/bitcoin' AND 
    toYear(created_at) >= '2019' AND 
    event_type = 'PullRequestEvent'
GROUP BY month, year
ORDER BY year DESC, month DESC
'''
'''
SELECT
    ROUND( SUM(opened) / COUNT(month), 2) AS average
FROM
(
    SELECT  
        SUM(action = 'opened') AS opened,
        toYear(created_at) AS year, 
        toMonth(created_at) AS month
    FROM github_events
    WHERE repo_name = 'bitcoin/bitcoin' AND 
        event_type = 'PullRequestEvent' AND 
        created_at >= dateSub(MONTH, 7,toStartOfMonth(now())) AND
        created_at < dateSub(MONTH, 4,toStartOfMonth(now()))
    GROUP BY month, year
    ORDER BY year DESC, month DESC
)
'''
query_L='''
SELECT
    ROUND( SUM(opened) / COUNT(month), 2) AS average
FROM
(
    SELECT  
        SUM(action = 'opened') AS opened,
        toYear(created_at) AS year, 
        toMonth(created_at) AS month
    FROM github_events
    WHERE repo_name = 
'''
query_R='''
AND 
        event_type = 'PullRequestEvent' AND 
        created_at >= dateSub(MONTH, 7,toStartOfMonth(now())) AND
        created_at < dateSub(MONTH, 4,toStartOfMonth(now()))
    GROUP BY month, year
    ORDER BY year DESC, month DESC
)
'''

In [56]:
# Read in 200_repos.csv
dfr = pd.read_csv('200_repos.csv', index_col=0)
# new df with only 2 columns
# 'CMC_id' as index is maintained
df = dfr[['repo','forge']].copy()
runQuery('PR_open', query_L, query_R,df)


...............................................................................................................................................................................................................................

'168 repos queried and 125 PR_open stats updated '

In [57]:
# update MERGED sheet with new data
# 'CMC_id' is the key, however 'repo', and 'forge' are also merged
# to prevent duplicate columns
df_temp = pd.read_csv('200_merged.csv', index_col = 0)
dfm = pd.merge(df_temp,df,on = ['CMC_id', 'repo', 'forge'])
dfm.to_csv('200_merged.csv', encoding = 'utf-8', index = 1)

In [227]:
dfm

,repo,forge,stars,forks,authors,commits,comments,PR_open
CMC_id,,,,,,,,
1,bitcoin/bitcoin,github,53646.0,26356.0,130.50,365.00,3107.33,154.67
1027,ethereum/go-ethereum,github,30598.0,12090.0,98.75,65.67,666.33,74.33
1839,binance-chain/bsc,github,55.0,25.0,8.00,8.00,9.00,4.50
825,NaN,private,NaN,NaN,NaN,NaN,NaN,NaN
2010,input-output-hk/cardano-node,github,357.0,139.0,43.25,1050.33,651.33,97.00
...,...,...,...,...,...,...,...,...
1993,NaN,other,NaN,NaN,NaN,NaN,NaN,NaN
541,syscoin,github,NaN,0.0,NaN,NaN,0.00,0.00
3714,legalthings,github,NaN,0.0,NaN,NaN,0.00,0.00


In [58]:
# udpate AVERAGE RESPONSE TIME and MEDIAN RESPONSE TIME
# 1. read in 200_repos.csv
# > keep columns: id, repo
# 2. build query
# 3. run query: update avg_resp_time; med_resp_time
# 4. write out 200_response_time.csv
# 5. make backup of 200_merged.csv
# 6. merge with 200_merged.csv

# 1. Read in 200_repos.csv
dfr = pd.read_csv('200_repos.csv', index_col=0)
# new df with only 2 columns
# 'CMC_id' as index is maintained
df = dfr[['repo','forge']].copy()

In [59]:
# 2. build query
#---------------------------------------
# A V E R A G E  C A L C U L A T I O N
#---------------------------------------
# timestamp set to 2019-11-01 to allow for a year of activity
#
query_avg_issue = '''
WITH repo_name = 'bitcoin/bitcoin' AS repo,
     sum(dateDiff('minute', toDateTime(opened), toDateTime(closed)))/60/24 AS total_days,
     round(dateDiff('second', toDateTime(opened), toDateTime(closed)),2)/60 AS mins_open, 
	 count() AS num_issues,
     (event_type = 'IssuesEvent' OR event_type = 'PullRequestEvent') AS event,
     created_at >= toDateTime('2019-11-01') AS created
SELECT  round( total_days / num_issues, 2) AS average_response_time_days
FROM
(
    SELECT *
    FROM
   (
        SELECT 	number,
		created_at AS opened
	FROM github_events 
	WHERE 	repo AND
		event AND
		action = 'opened' AND
		created
    ) AS t1
    INNER JOIN
    (
        SELECT 	number,
		created_at AS closed
	FROM github_events 
	WHERE 	repo AND
		event AND
		action = 'closed' AND
		created
    ) AS t2 USING (number)
)
WHERE mins_open > 5
'''
query_avg_issue_L = '''
WITH repo_name =
'''
query_avg_issue_R = '''
AS repo,
     sum(dateDiff('minute', toDateTime(opened), toDateTime(closed)))/60/24 AS total_days,
     round(dateDiff('second', toDateTime(opened), toDateTime(closed)),2)/60 AS mins_open, 
	 count() AS num_issues,
     (event_type = 'IssuesEvent' OR event_type = 'PullRequestEvent') AS event,
     created_at >= toDateTime('2019-11-01') AS created
SELECT  round( total_days / num_issues, 2) AS average_response_time_days
FROM
(
    SELECT *
    FROM
   (
        SELECT 	number,
		created_at AS opened
	FROM github_events 
	WHERE 	repo AND
		event AND
		action = 'opened' AND
		created
    ) AS t1
    INNER JOIN
    (
        SELECT 	number,
		created_at AS closed
	FROM github_events 
	WHERE 	repo AND
		event AND
		action = 'closed' AND
		created
    ) AS t2 USING (number)
)
WHERE mins_open > 5
'''
#
#---------------------------------------
# M E D I A N    C A L C U L A T I O N
#---------------------------------------
# timestamp set to 2019-11-01 to allow for a year of activity
#
query_med_issue = '''
WITH repo_name = 'bitcoin/bitcoin' AS repo,
     round(dateDiff('second', toDateTime(opened), toDateTime(closed)),2)/60 AS mins_open,
     round(dateDiff('second', toDateTime(opened), toDateTime(closed)),2)/60/60/24 AS days_open, 
     (event_type = 'IssuesEvent' OR event_type = 'PullRequestEvent') AS event,
     created_at >= toDateTime('2019-11-01') AS created
SELECT  round(medianDeterministic(days_open, 1),2) as median_response_time_days
FROM
(
    SELECT *
    FROM
   (
        SELECT 	number,
		created_at AS opened
	FROM github_events 
	WHERE 	repo AND
		event AND
		action = 'opened' AND
		created
    ) AS t1
    INNER JOIN
    (
        SELECT 	number,
		created_at AS closed
	FROM github_events 
	WHERE 	repo AND
		event AND
		action = 'closed' AND
		created
    ) AS t2 USING (number)
)
WHERE mins_open > 5
'''
query_med_issue_L = '''
WITH repo_name =
'''
query_med_issue_R = '''
AS repo,
     round(dateDiff('second', toDateTime(opened), toDateTime(closed)),2)/60 AS mins_open,
     round(dateDiff('second', toDateTime(opened), toDateTime(closed)),2)/60/60/24 AS days_open, 
     (event_type = 'IssuesEvent' OR event_type = 'PullRequestEvent') AS event,
     created_at >= toDateTime('2019-11-01') AS created
SELECT  round(medianDeterministic(days_open, 1),2) as median_response_time_days
FROM
(
    SELECT *
    FROM
   (
        SELECT 	number,
		created_at AS opened
	FROM github_events 
	WHERE 	repo AND
		event AND
		action = 'opened' AND
		created
    ) AS t1
    INNER JOIN
    (
        SELECT 	number,
		created_at AS closed
	FROM github_events 
	WHERE 	repo AND
		event AND
		action = 'closed' AND
		created
    ) AS t2 USING (number)
)
WHERE mins_open > 5
'''

In [60]:
# 3. run query: update avg_resp_time; med_resp_time
runQuery('avg_resp_time', query_avg_issue_L, query_avg_issue_R, df)


...............................................................................................................................................................................................................................

'168 repos queried and 131 avg_resp_time stats updated '

In [61]:
runQuery('med_resp_time', query_med_issue_L, query_med_issue_R, df)

...............................................................................................................................................................................................................................

'168 repos queried and 131 med_resp_time stats updated '

In [62]:
# 4. write out 200_response_time.csv
df.to_csv('200_response_time.csv', encoding='utf-8', index=1)


In [63]:
# 5. make backup of 200_merged.csv
df_temp = pd.read_csv('200_merged.csv', index_col=0)
df_temp.to_csv('200_merged_backup.csv', encoding='utf-8', index=1)

In [64]:
# 6. merge with 200_merged.csv
# update MERGED sheet with new data
# 'CMC_id' is the key, drop 'repo', and 'forge' before the merge
# to prevent duplicate columns
df.drop(columns = ['repo','forge'], inplace = True)
dfm = pd.merge(df_temp,df,on = ['CMC_id'], how = 'outer')


In [65]:
# 7. write out new merge
dfm.to_csv('200_merged.csv', encoding='utf-8', index = 1)

In [66]:
# udpate AVGERAGE LONGEVITY of contributors
# 1. read in 200_repos.csv
# > keep columns: id, repo
# 2. build query
# 3. run query: update avg_longevity;
# 4. write out 200_longevity.csv
# 5. make backup of 200_merged.csv
# 6. merge with 200_merged.csv

# 1. Read in 200_repos.csv
dfr = pd.read_csv('200_repos.csv', index_col=0)
# new df with only 2 columns
# 'CMC_id' as index is maintained
df = dfr[['repo','forge']].copy()

In [67]:
# 2. build query
#---------------------------------------
# L O N G E V I T Y  C A L C U L A T I O N
#---------------------------------------
# searches the repo for all activity by a contributor
# excludes those that step in 'one-time'
# such as starring or forking or leaving a comment and never returning
# this excludes devs that work on other projects 
# (only calculates from single repo)
#---------------------------------------
# view multiple author's days active
#---------------------------------------
'''
WITH dateDiff('day', toDateTime(earliest_seen), toDateTime(last_seen)) AS days_active
SELECT days_active, 
	actor_login, 
	last_seen
FROM
(  	SELECT MIN(created_at) AS earliest_seen, 
			MAX(created_at) AS last_seen, 
			days_active, 
			actor_login
		FROM github_events 
		WHERE repo_name = 'bitcoin/bitcoin'
		GROUP by actor_login
		ORDER by days_active DESC
)
WHERE days_active > 0
'''
#
#---------------------------------------
# calculate A V G.  D E V.  L O N G E V I T Y
#---------------------------------------
'''
WITH dateDiff('day', toDateTime(earliest_seen), 
	toDateTime(last_seen)) AS days_active
SELECT ROUND((SUM(days_active) / count() ),2) AS avg_dev_days_active
FROM
(  SELECT MIN(created_at) AS earliest_seen, 
		MAX(created_at) AS last_seen, 
		days_active, 
		actor_login, 
		count()
   FROM github_events 
   WHERE repo_name = 'bitcoin/bitcoin'
   GROUP by actor_login
   ORDER by days_active DESC
)
WHERE days_active > 0
'''
query_avg_long_L = '''
WITH dateDiff('day', toDateTime(earliest_seen), 
	toDateTime(last_seen)) AS days_active
SELECT ROUND((SUM(days_active) / count() ),2) AS avg_dev_days_active
FROM
(  SELECT MIN(created_at) AS earliest_seen, 
		MAX(created_at) AS last_seen, 
		days_active, 
		actor_login, 
		count()
   FROM github_events 
   WHERE repo_name = 
'''
query_avg_long_R = '''
GROUP by actor_login
   ORDER by days_active DESC
)
WHERE days_active > 0
'''

In [68]:
# 3. run query: update avg_longevity
runQuery('avg_longevity_days', query_avg_long_L, query_avg_long_R, df)

...............................................................................................................................................................................................................................

'168 repos queried and 140 avg_longevity_days stats updated '

In [33]:
df

,repo,forge,avg_longevity_days
CMC_id,,,
1,bitcoin/bitcoin,github,390.51
1027,ethereum/go-ethereum,github,243.06
1839,binance-chain/bsc,github,54.86
825,NaN,private,NaN
2010,input-output-hk/cardano-node,github,139.99
...,...,...,...
1993,NaN,other,NaN
541,syscoin,github,0.00
3714,legalthings,github,0.00


In [69]:
# 4. write out 200_longevity.csv
df.to_csv('200_longevity.csv', encoding='utf-8', index=1)

In [70]:
# 5. make backup of 200_merged.csv
df_temp = pd.read_csv('200_merged.csv', index_col=0)
df_temp.to_csv('200_merged_backup.csv', encoding='utf-8', index=1)

In [71]:
# 6. merge with 200_merged.csv
# update MERGED sheet with new data
# 'CMC_id' is the key, drop 'repo', and 'forge' before the merge
# to prevent duplicate columns
df.drop(columns = ['repo','forge'], inplace = True)
dfm = pd.merge(df_temp,df,on = ['CMC_id'], how = 'outer')


In [72]:
# 7. write out new merge
dfm.to_csv('200_merged.csv', encoding='utf-8', index = 1)